In [ ]:
from langchain_community.document_loaders import TextLoader
# convert the text so langchain can work with
from langchain_text_splitters import CharacterTextSplitter

from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [23]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
import pandas as pd

books = pd.read_csv("data/books_cleaned.csv")

books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...


In [ ]:
# Create a file with tagged descriptions
with open("tagged_descriptions.txt", "w") as f:
    f.write("\n".join(books["tagged_description"]))


In [ ]:

raw_docs = TextLoader("tagged_descriptions.txt").load()
# Get each book description as a document. (Split by new line)
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_docs)

Created a chunk of size 1168, which is longer than the specified 0
Created a chunk of size 1214, which is longer than the specified 0
Created a chunk of size 373, which is longer than the specified 0
Created a chunk of size 309, which is longer than the specified 0
Created a chunk of size 479, which is longer than the specified 0
Created a chunk of size 482, which is longer than the specified 0
Created a chunk of size 960, which is longer than the specified 0
Created a chunk of size 188, which is longer than the specified 0
Created a chunk of size 843, which is longer than the specified 0
Created a chunk of size 284, which is longer than the specified 0
Created a chunk of size 193, which is longer than the specified 0
Created a chunk of size 877, which is longer than the specified 0
Created a chunk of size 1088, which is longer than the specified 0
Created a chunk of size 1189, which is longer than the specified 0
Created a chunk of size 304, which is longer than the specified 0
Create

In [6]:
documents[0]

Document(metadata={'source': 'tagged_descriptions.txt'}, page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, G

In [ ]:
# Create a Chroma vector database, embed the documents using OpenAI embeddings, and persist the database.
db_books = Chroma.from_documents(documents, embedding=OpenAIEmbeddings(), persist_directory="./chroma_db")

In [8]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k=10)

In [9]:
docs

[Document(id='8e0b1beb-1d72-4699-88d0-8693a5228e16', metadata={'source': 'tagged_descriptions.txt'}, page_content='9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.'),
 Document(id='72bff8ce-de76-4250-8f90-a4ae7417c5eb', metadata={'source': 'tagged_descriptions.txt'}, page_content="9780786808380 Introduce your babies to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child's drawings and abstract art to playful photos. A brief text accompanies each image, introducing the baby to some basic -- and sometimes playful -- information about the subjects."),
 Document(id='1168d1b5-5bfc-44ac-a6c4-beb9ab7c41a9', metadata={'source':

In [28]:
books[books["isbn13"] == int(docs[0].page_content.split()[0].strip())]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_subtitle,tagged_description
3747,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...


In [10]:

def retrieve_semantic_recommendations(
    query: str,
    top_k: int = 10,
) -> pd.DataFrame:
    """
    Retrieve semantic recommendations based on a query.

    Args:
        query (str): The query to search for.
        top_k (int, optional): The number of recommendations to retrieve. Defaults to 10.

    Returns:
        pd.DataFrame: A DataFrame containing the recommendations.
    """
    recs = db_books.similarity_search(query, k=top_k)
    books_list = []

    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)]

In [11]:
retrieve_semantic_recommendations("A book to teach children about nature")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_subtitle,tagged_description
442,9780067575208,006757520X,The Sense of Wonder,Rachel Carson,Nature,http://books.google.com/books/content?id=Zee5S...,"First published more than three decades ago, t...",1998.0,4.39,112.0,1160.0,The Sense of Wonder,9780067575208 First published more than three ...
3214,9780689861130,0689861133,"Moo, Baa, la la La!",Sandra Boynton,Animal sounds,http://books.google.com/books/content?id=Gz40A...,Children will love joining in and imitating th...,2004.0,4.20,14.0,28261.0,"Moo, Baa, la la La!",9780689861130 Children will love joining in an...
3581,9780763620875,0763620874,Judy Moody Saves the World!,Megan McDonald,Juvenile Fiction,http://books.google.com/books/content?id=xDIRB...,When Judy Moody gets serious about protecting ...,2004.0,4.03,160.0,5883.0,Judy Moody Saves the World!,9780763620875 When Judy Moody gets serious abo...
3747,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...
3748,9780786808373,0786808373,Baby Einstein: Birds,Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=0jxHP...,"Introducing your baby to birds, cats, dogs, an...",2002.0,3.78,20.0,9.0,Baby Einstein: Birds,"9780786808373 Introducing your baby to birds, ..."
3749,9780786808380,0786808381,Baby Einstein: Babies,Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=jv4NA...,"Introduce your babies to birds, cats, dogs, an...",2002.0,4.03,20.0,29.0,Baby Einstein: Babies,"9780786808380 Introduce your babies to birds, ..."
3750,9780786808397,078680839X,Baby Einstein: Dogs,Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=qut8t...,"Introduce your baby to birds, cats, dogs, and ...",2002.0,3.81,20.0,26.0,Baby Einstein: Dogs,"9780786808397 Introduce your baby to birds, ca..."
3751,9780786808717,0786808713,Baby Einstein: What Does Violet See? Raindrops...,Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=95IIA...,A very special puddle sets Violet the mouse of...,2002.0,3.25,18.0,16.0,Baby Einstein: What Does Violet See? Raindrops...,9780786808717 A very special puddle sets Viole...
3760,9780786812912,0786812915,The Big Box,Toni Morrison;Slade Morrison,Juvenile Fiction,http://books.google.com/books/content?id=LyYKA...,"In her first illustrated book for children, th...",2002.0,3.95,48.0,375.0,The Big Box,9780786812912 In her first illustrated book fo...
3797,9780789458209,0789458209,Tree,David Burnie,Juvenile Nonfiction,http://books.google.com/books/content?id=Qwsqj...,Photographs and text explore the anatomy and l...,2000.0,4.07,64.0,5.0,Tree,9780789458209 Photographs and text explore the...


In [ ]:
books["categories"].value_counts().reset_index().query("count > 50")

category_mapping = {
    # Fiction Categories
    'Fiction': 'Fiction',
    'Drama': 'Fiction',
    'Literary Criticism': 'Fiction',
    'Poetry': 'Fiction',
    'Comics & Graphic Novels': 'Fiction',
    
    # Nonfiction Categories
    'Biography & Autobiography': 'Nonfiction',
    'History': 'Nonfiction',
    'Philosophy': 'Nonfiction',
    'Religion': 'Nonfiction',
    'Science': 'Nonfiction',
    
    # Children's Nonfiction Categories
    'Juvenile Nonfiction': "Children's Nonfiction",
}

In [13]:
books["simple_categories"] = books["categories"].map(category_mapping)
books[~(books["simple_categories"].isna())]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_subtitle,tagged_description,simple_categories
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction
8,9780006482079,0006482074,Warhost of Vastmark,Janny Wurts,Fiction,http://books.google.com/books/content?id=uOL0f...,"Tricked once more by his wily half-brother, Ly...",1995.0,4.03,522.0,2966.0,Warhost of Vastmark,9780006482079 Tricked once more by his wily ha...,Fiction
30,9780006646006,000664600X,Ocean Star Express,Mark Haddon;Peter Sutton,Juvenile Fiction,http://books.google.com/books/content?id=I2QZA...,Joe and his parents are enjoying a summer holi...,2002.0,3.50,32.0,1.0,Ocean Star Express,9780006646006 Joe and his parents are enjoying...,Children's Fiction
46,9780007121014,0007121016,Taken at the Flood,Agatha Christie,Fiction,http://books.google.com/books/content?id=3gWlx...,A Few Weeks After Marrying An Attractive Young...,2002.0,3.71,352.0,8852.0,Taken at the Flood,9780007121014 A Few Weeks After Marrying An At...,Fiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5178,9781933648279,1933648279,Night Has a Thousand Eyes,Cornell Woolrich,Fiction,http://books.google.com/books/content?id=3Gk6s...,"""Cornell Woolrich's novels define the essence ...",2007.0,3.77,344.0,680.0,Night Has a Thousand Eyes,"9781933648279 ""Cornell Woolrich's novels defin...",Fiction
5188,9784770028969,4770028962,Coin Locker Babies,村上龍,Fiction,http://books.google.com/books/content?id=87DJw...,Rescued from the lockers in which they were le...,2002.0,3.75,393.0,5560.0,Coin Locker Babies,9784770028969 Rescued from the lockers in whic...,Fiction
5189,9788122200850,8122200850,"Cry, the Peacock",Anita Desai,Fiction,http://books.google.com/books/content?id=_QKwV...,This book is the story of a young girl obsesse...,1980.0,3.22,218.0,134.0,"Cry, the Peacock",9788122200850 This book is the story of a youn...,Fiction
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction


In [ ]:
from transformers import pipeline

fiction_categories = ["Fiction", "Nonfiction"]

pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device="mps")

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use mps


In [ ]:
sequence = books.loc[books["simple_categories"] == "Fiction", "description"].reset_index(drop=True)[0]
sequence

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [ ]:
pipe(sequence, fiction_categories)

{'sequence': 'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst

In [20]:
import numpy as np
max_index = np.argmax(pipe(sequence, fiction_categories)["scores"])
max_label = pipe(sequence, fiction_categories)["labels"][max_index]
max_label

'Fiction'

In [21]:
def generate_prediction(sequence, categories):
     predictions = pipe(sequence, fiction_categories)
     max_index = np.argmax(predictions["scores"])
     max_label = predictions["labels"][max_index]
     return max_label


In [22]:
from tqdm import tqdm

actual_cats = []
predicted_cats = []

for i in tqdm(range(0, 300)):
     sequence = books.loc[books["simple_categories"] == "Fiction", "description"].reset_index(drop=True)[i]
     predicted_cats += [generate_prediction(sequence, fiction_categories)]
     actual_cats += ["Fiction"]

100%|██████████| 300/300 [01:20<00:00,  3.72it/s]


In [23]:
for i in tqdm(range(0, 300)):
     sequence = books.loc[books["simple_categories"] == "Nonfiction", "description"].reset_index(drop=True)[i]
     predicted_cats += [generate_prediction(sequence, fiction_categories)]
     actual_cats += ["Nonfiction"]

100%|██████████| 300/300 [00:59<00:00,  5.03it/s]


In [26]:
prediction_df = pd.DataFrame({"actual_categories": actual_cats, "predicted_categories": predicted_cats})
prediction_df["correct_prediction"] = np.where(prediction_df["actual_categories"] == prediction_df["predicted_categories"], 1, 0)
prediction_df["correct_prediction"].sum() / len(prediction_df)

0.7833333333333333

In [36]:
isbns = []
predicted_cats = []

missing_cats = books.loc[books['simple_categories'].isna(), ["isbn13", "description"]].reset_index(drop=True)
missing_cats

,isbn13,description
0,9780002261982,A new 'Christie for Christmas' -- a full-lengt...
1,9780006280897,Lewis' work on the nature of love divides love...
2,9780006280934,"""In The Problem of Pain, C.S. Lewis, one of th..."
3,9780006380832,Until Vasco da Gama discovered the sea-route t...
4,9780006470229,A new-cover reissue of the fourth book in the ...
...,...,...
1449,9788125026600,Not only does Nietzsche for Beginners delve in...
1450,9788171565641,"Forster's lively, informed originality and wit..."
1451,9788172235222,On A Train Journey Home To North India After L...
1452,9788173031014,This book tells the tale of a man who goes on ...


In [37]:
for i in tqdm(range(0, len(missing_cats))):
     sequence = missing_cats["description"][i]
     predicted_cats += [generate_prediction(sequence, fiction_categories)]
     isbns += [missing_cats["isbn13"][i]]



100%|██████████| 1454/1454 [03:02<00:00,  7.95it/s]


In [38]:
predicted_cats

['Fiction',
 'Nonfiction',
 'Nonfiction',
 'Nonfiction',
 'Fiction',
 'Nonfiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Nonfiction',
 'Nonfiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Nonfiction',
 'Nonfiction',
 'Nonfiction',
 'Fiction',
 'Nonfiction',
 'Nonfiction',
 'Nonfiction',
 'Nonfiction',
 'Nonfiction',
 'Fiction',
 'Nonfiction',
 'Fiction',
 'Nonfiction',
 'Nonfiction',
 'Nonfiction',
 'Fiction',
 'Fiction',
 'Nonfiction',
 'Nonfiction',
 'Nonfiction',
 'Nonfiction',
 'Nonfiction',
 'Nonfiction',
 'Nonfiction',
 'Nonfiction',
 'Fiction',
 'Nonfiction',
 'Nonfiction',
 'Nonfiction',
 'Nonfiction',
 'Fiction',
 'Fiction',
 'Nonfiction',
 'Fiction',
 'Fiction',
 'Nonfiction',
 'Nonfiction',
 'Nonfiction',
 'Fiction',
 'Nonfiction',
 'Fiction',
 'Fiction',
 'Nonfiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Fiction',
 'Nonfiction',
 'Nonfiction',
 'Fiction',
 'Nonfiction',
 'Fiction',
 'Nonfiction',
 'Nonfiction',
 

In [39]:
missing_prediction_df = pd.DataFrame({"isbn13": isbns, "predicted_categories": predicted_cats})


In [40]:
books = pd.merge(books, missing_prediction_df, on= "isbn13", how="left")
books["simple_categories"] = np.where(books["simple_categories"].isna(), books["predicted_categories"], books["simple_categories"])
books = books.drop(columns=["predicted_categories"])

In [43]:
books
books["simple_categories"].isna().sum()

0

In [ ]:
books.to_csv("data/books_with_categories.csv", index= False)